In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



In [29]:
import requests
import pandas as pd
import config
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from assignment1 import *
import re
import time
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
client = Socrata("data.cityofchicago.org",config.key)
conn = sqlite3.connect('db.db')
c = conn.cursor()
from pandas import read_sql_query as pq

## Scope of Analysis, Summer 2016
In the following analyses, we use data from the following datasets, focussing specifically on the months of June, July, and August of 2016:
- [311 Graffiti Removal Requests](https://data.cityofchicago.org/Service-Requests/311-Service-Requests-Graffiti-Removal/hec5-y4x5)
- [311 Vacant and Abandoned Buildings Requests](https://data.cityofchicago.org/Service-Requests/311-Service-Requests-Vacant-and-Abandoned-Building/7nii-7srd)
- [311 Alley Light Out Requests](https://data.cityofchicago.org/Service-Requests/311-Service-Requests-Alley-Lights-Out/t28b-ys7j)
- [American Community Survey 2016, US Census](https://www.census.gov/data/developers/data-sets/acs-5year.html)

## Initial Data Exploration

## Service Request Counts for Summer 2016
In June, July, and August of 2016, there were significantly more Graffiti removal requests compared to Alley Light Out and Vacant/Abandoned Buildings Reports. 

In [79]:
barplot(pd.read_sql_query('''
SELECT type_of_service_request,
COUNT(DISTINCT service_request_number) as Count
FROM combined_data
GROUP BY type_of_service_request
;''',conn),
x = 'type_of_service_request',
y = 'Count', 
xlabel = 'Service Request Type', 
ylabel ='Service Request Type', 
title = "Service Request Type Counts for Summer 2016")

## Service Requests by Month
There were more service requests in August 2016 across these three categories compared to June and July.

In [52]:
r = pq('''
SELECT type_of_service_request, strftime('%m',creation_date) as month,
COUNT(DISTINCT service_request_number) as Count
FROM combined_data
GROUP BY type_of_service_request, month''', conn)
trace1 = go.Bar(
    x=['June 2016', 'July 2016', 'August 2016'],
    y=list(r.Count[r.type_of_service_request == 'Alley Light Out']),
    name='Alley Light Out'
)
trace2 = go.Bar(
    x=['June 2016', 'July 2016', 'August 2016'],
    y=list(r.Count[r.type_of_service_request == 'Graffiti Removal']),
    name='Graffiti Removal'
)
trace3 = go.Bar(
    x=['June 2016', 'July 2016', 'August 2016'],
    y=list(r.Count[r.type_of_service_request == 'Vacant/Abandoned Building']),
    name='Vacant/Abandoned Building'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='stack',
    title='<b>Service Requests by Month</b><br>',
             xaxis=dict(
                title= 'Month',
                tickfont=dict(
                    size=14,
                    color='rgb(107, 107, 107)'
                )
             ),
             yaxis=dict(
                title='Service Request Count',
                titlefont=dict(
                    size=16,
                    color='rgb(107, 107, 107)'
                ),
                tickfont=dict(
                    size=14,
                    color='rgb(107, 107, 107)'
                )
             ),
            )

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

In [78]:
barplot(pq('''
SELECT b.name as community,
COUNT(DISTINCT service_request_number) as Count
FROM combined_data a
JOIN community_areas b
on a.community_area = b.community_area
GROUP BY community
ORDER BY 2 DESC''', conn),
x = 'community',
y = 'Count',
xlabel = 'Community Area',
ylabel = 'Service Request Count',
title = 'Service Requests by Community Area',
xsize = 9)